<a href="https://colab.research.google.com/github/yogeshrana7/BDDA2/blob/main/credit_card_fraud_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [63]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score, classification_report
from  sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
file_transaction = pd.read_csv("/content/drive/MyDrive/creditcard.csv")
file_transaction.head(5)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [66]:
file_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [67]:
file_transaction.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [68]:
file_transaction['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [69]:
file_legit_transaction = file_transaction[file_transaction.Class == 0] 
file_fraud_transaction = file_transaction[file_transaction.Class == 1]
#print(file_fraud_transaction.to_string()) # print the new sub table

In [70]:
print(file_legit_transaction.shape)
print(file_fraud_transaction.shape)

(284315, 31)
(492, 31)


In [71]:
file_legit_transaction.Amount.describe()

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [72]:
file_fraud_transaction.Amount.describe()

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [73]:
file_transaction.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,0.009824,-0.006576,0.010832,0.000189,0.012064,0.000161,0.007164,0.011535,0.003887,-0.001178,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [74]:
file_legit_transaction_sample = file_legit_transaction.sample(n=492)

In [75]:
new_file_transaction= pd.concat([file_legit_transaction_sample, file_fraud_transaction], axis = 0)

In [76]:
new_file_transaction.shape

(984, 31)

In [77]:
new_file_transaction.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
271776,164741.0,-0.923377,1.752366,-0.494198,-0.481745,0.685165,-0.611919,0.698739,0.312480,-0.257231,-0.668628,-0.986686,-0.163908,0.069363,-0.729039,0.075846,0.210828,0.678691,-0.024472,0.458715,0.143570,-0.305827,-0.837680,0.002063,0.378431,-0.289676,0.234282,0.184999,0.235718,1.79,0
171064,120483.0,-0.632781,-1.511681,2.758904,-2.504170,-1.221752,0.768253,-1.731258,0.521174,-0.688508,0.327463,-0.588297,0.585330,1.389595,-2.057836,-3.478976,-0.416665,0.281036,0.783544,-0.096999,-0.228192,0.074048,0.954574,-0.214308,-0.273706,-0.168501,-0.078714,0.218293,0.152654,10.30,0
283728,171829.0,2.070626,0.176873,-1.763674,0.381060,0.543542,-0.772793,0.238878,-0.266992,0.401706,-0.401566,-0.592575,0.453771,0.767034,-0.986030,0.143743,0.331473,0.378399,-0.337689,0.104613,-0.090962,-0.362166,-0.922595,0.305145,0.397790,-0.217469,0.178212,-0.060051,-0.029157,9.99,0
224172,143681.0,2.228097,-0.609542,-2.733459,-1.133327,0.587938,-0.904955,0.334597,-0.415603,-1.244840,1.095342,0.603852,-0.348856,-0.510095,0.691161,-1.075710,0.355323,0.220389,-1.143698,1.117963,0.003349,0.516540,1.434135,-0.266728,0.384155,0.766139,0.273868,-0.109445,-0.096551,40.00,0


In [78]:
new_file_transaction['Class'].value_counts()

1    492
0    492
Name: Class, dtype: int64

In [79]:
new_file_transaction.groupby('Class').mean()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,91719.136179,-0.035207,-0.103643,0.011012,-0.055423,-0.003151,0.037009,-0.028866,0.038628,0.032827,0.040505,-0.036936,0.070688,-0.056821,-0.047686,0.048625,-0.010911,-0.003476,0.036493,0.028934,-0.008335,-0.005245,0.003103,-0.014094,0.013263,0.043323,0.003560,0.014390,-0.021507,92.710346
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,-5.676883,3.800173,-6.259393,-0.109334,-6.971723,-0.092929,-4.139946,-6.665836,-2.246308,0.680659,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [80]:
X = new_file_transaction.drop(columns = 'Class', axis=1)
Y = new_file_transaction['Class']

In [81]:
X.shape

(984, 30)

In [82]:
Y.shape

(984,)

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)


In [90]:
model = LogisticRegression()

In [91]:
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [92]:
train_predictions = model.predict(X_train)
train_accuracy = accuracy_score(train_predictions, y_train)
print(train_accuracy)

0.9529860228716646


In [93]:
test_predictions = model.predict(X_test)
test_accuracy = accuracy_score(test_predictions, y_test)
print(test_accuracy)

0.9086294416243654


In [94]:
print(classification_report(y_test,test_predictions))

              precision    recall  f1-score   support

           0       0.88      0.95      0.91        99
           1       0.94      0.87      0.90        98

    accuracy                           0.91       197
   macro avg       0.91      0.91      0.91       197
weighted avg       0.91      0.91      0.91       197



#####  Conclusion : Model have achieved an accuracy of approx 91% .